### Transformer BERT GPT2 예제
[Colab에서 실행]

In [1]:
# ! pip install torch torchvision

import torch
import torch.nn as nn
import torch.optim as optim

# 신경망 모델 정의
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# 모델, 손실 함수, 옵티마이저 정의
model = SimpleNN(input_size=3, hidden_size=5, output_size=2)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 예제 입력과 타겟 데이터
inputs = torch.tensor([[1.0, 2.0, 3.0]])
targets = torch.tensor([[0.0, 1.0]])

# 모델 학습
for epoch in range(100):
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

# 모델 평가
with torch.no_grad():
    outputs = model(inputs)
    predicted = torch.argmax(outputs, dim=1)
    print(f'Predicted: {predicted}')

Epoch [10/100], Loss: 0.9587
Epoch [20/100], Loss: 0.5575
Epoch [30/100], Loss: 0.3296
Epoch [40/100], Loss: 0.1702
Epoch [50/100], Loss: 0.0730
Epoch [60/100], Loss: 0.0267
Epoch [70/100], Loss: 0.0090
Epoch [80/100], Loss: 0.0030
Epoch [90/100], Loss: 0.0010
Epoch [100/100], Loss: 0.0004
Predicted: tensor([1])


### Transformer
: 구글이 2017년에 발표한 자연어 처리(NLP) 모델로, 그 이후로 많은 NLP 작업에서 획기적인 성능 향상을 이룬 모델이다. Transformer의 주요 혁신은 기존의 순환 신경망(RNN)이나 LSTM(Long Short-Term Memory)과 달리, 시퀀스를 처리할 때 순차적인 계산이 필요 없다는 점이다. 이로 인해 병렬 처리가 가능해졌고, 더 큰 모델을 효과적으로 학습할 수 있다

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Transformer 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, nhead, num_encoder_layers, num_decoder_layers, output_dim):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Embedding(input_dim, model_dim)
        self.transformer = nn.Transformer(
            d_model=model_dim,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers
        )
        self.fc_out = nn.Linear(model_dim, output_dim)

    def forward(self, src, tgt):
        src = self.embedding(src)  # (src_seq_len, N, model_dim)
        tgt = self.embedding(tgt)  # (tgt_seq_len, N, model_dim)

        # Transformer expects (seq_len, batch_size, model_dim)
        output = self.transformer(src, tgt)
        output = self.fc_out(output)
        return output

# 하이퍼파라미터 설정
input_dim = 10  # 예시 입력 차원 (어휘 크기)
model_dim = 512  # 모델 차원
nhead = 8  # 멀티헤드 어텐션의 헤드 수
num_encoder_layers = 3  # 인코더 레이어 수
num_decoder_layers = 3  # 디코더 레이어 수
output_dim = 10  # 예시 출력 차원 (어휘 크기)

# 모델 인스턴스 생성
model = TransformerModel(input_dim, model_dim, nhead, num_encoder_layers, num_decoder_layers, output_dim)

# 입력 및 출력 시퀀스 예시 (텐서로 변환)
src = torch.randint(0, input_dim, (5, 1))  # (src_seq_len, batch_size)
tgt = torch.randint(0, input_dim, (5, 1))  # (tgt_seq_len, batch_size)

# 모델 출력 계산
output = model(src, tgt)

print("Model output shape:", output.shape)
print("Model output:", output)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Model output shape: torch.Size([5, 1, 10])
Model output: tensor([[[ 0.1498,  1.3707, -0.6535,  0.3827,  0.2009,  0.2919, -0.5569,
           0.1602, -0.0967, -0.0332]],

        [[ 1.0357,  1.0251, -0.7247, -0.0464,  0.1237,  0.0134,  0.2956,
          -0.2458,  0.6371, -0.4632]],

        [[ 0.7356, -0.0029, -1.2712, -0.0243,  0.0728,  0.6858,  0.6377,
          -0.0541,  0.0603, -0.4277]],

        [[ 0.9750,  1.2079, -0.0882, -0.4421, -0.3538,  0.4816, -0.0931,
          -0.1144,  0.0105, -0.3523]],

        [[ 0.9583,  1.4565, -1.3288, -0.5301,  0.8749,  0.8493,  0.4207,
          -0.2496,  0.0946, -0.5999]]], grad_fn=<ViewBackward0>)


### BERT(Bidirectional Encoder Representations from Transformers)
: Google에서 개발한 자연어 처리(NLP) 모델로, 다양한 NLP 작업에서 혁신적인 성능을 보여주는 모델이다. BERT는 주어진 텍스트의 문맥을 양방향으로 이해할 수 있도록 설계된 최초의 모델 중 하나이다.

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

# 1. BERT 모델과 토크나이저 로드
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 2. 예제 문장 정의
sentences = ["I love this movie!", "This is the worst product I have ever bought."]

# 3. 입력 텍스트를 BERT 모델 입력 형식으로 변환
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

# 4. 모델에 입력 데이터 전달하여 예측 수행
with torch.no_grad():
    outputs = model(**inputs)

# 5. 결과 해석
logits = outputs.logits
probs = softmax(logits, dim=1)
predicted_class = torch.argmax(probs, dim=1)

# 클래스 레이블 (BERT 모델에 따라 달라질 수 있음)
class_labels = ["very negative", "negative", "neutral", "positive", "very positive"]

# 6. 예측 결과 출력
for sentence, pred in zip(sentences, predicted_class):
    print(f"Sentence: '{sentence}' is classified as '{class_labels[pred]}'")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Sentence: 'I love this movie!' is classified as 'very positive'
Sentence: 'This is the worst product I have ever bought.' is classified as 'very negative'


## GPT-2(GPT-2, Generative Pre-trained Transformer 2)는
: OpenAI에서 개발한 대규모 언어 모델로, 2019년에 발표.
Transformer 아키텍처를 기반. Transformer는 병렬 처리가 가능하고, 긴 문맥을 잘 처리할 수 있어 NLP 작업에서 매우 효과적이다.이 모델은 자연어 생성, 번역, 요약 등 다양한 자연어 처리 작업에서 좋은 성능을 보여주었다.

In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 1. GPT-2 모델과 토크나이저 로드
model_name = "gpt2"  # 또는 "gpt2-medium", "gpt2-large", "gpt2-xl" 등
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 2. 텍스트 프롬프트 정의
prompt = "Once upon a time"

# 3. 입력 텍스트를 GPT-2 모델 입력 형식으로 변환
inputs = tokenizer(prompt, return_tensors="pt")

# 4. 텍스트 생성
output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    max_length=50,  # 생성할 텍스트의 최대 길이
    temperature=0.7,  # 텍스트의 다양성 조절 (낮은 값: 더 예측 가능, 높은 값: 더 무작위)
    top_k=50,  # 상위 k개 단어에서 샘플링
    top_p=0.95,  # 상위 p% 확률 분포에서 샘플링 (nucleus sampling)
    do_sample=True,  # 샘플링 사용 여부
)

# 5. 결과 해석 및 출력
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print('-----------------')
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


-----------------
Once upon a time, it seems to be the most reliable way to start a conversation.

It's also the one way to get an idea of how many people have been talking to you.

It's also one way to get to


In [6]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 1. GPT-2 모델과 토크나이저 로드
model_name = "gpt2"  # 또는 "gpt2-medium", "gpt2-large", "gpt2-xl"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 2. 요약할 긴 텍스트 입력
text_to_summarize = (
    #"GPT-2 is a large-scale unsupervised language model that generates coherent paragraphs of text. "
    "It is the successor of GPT, developed by OpenAI. GPT-2 is capable of generating realistic and coherent text in a "
    "variety of styles and for a range of applications, without needing task-specific training data."
)


# 3. 요약을 유도하는 프롬프트 정의
prompt = "Summarize the following text:\n\n" + text_to_summarize + "\n\nSummary:"

# 4. 입력 텍스트를 GPT-2 모델 입력 형식으로 변환
inputs = tokenizer(prompt, return_tensors="pt")

# 5. 텍스트 생성 (요약)
output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    max_length=100,  # 생성할 텍스트의 최대 길이
    temperature=0.2,  # 텍스트의 다양성 조절
    top_k=50,  # 상위 k개 단어에서 샘플링
    top_p=0.95,  # 상위 p% 확률 분포에서 샘플링
    do_sample=True,  # 샘플링 사용 여부
    num_return_sequences=1  # 반환할 요약 문장 수
)

# 6. 결과 해석 및 출력
summary = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print("Summary:", summary)  # 성능이 좋지 않음, 영어위주로 학습되어 한글 지원 미비


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summary: Summarize the following text:

It is the successor of GPT, developed by OpenAI. GPT-2 is capable of generating realistic and coherent text in a variety of styles and for a range of applications, without needing task-specific training data.

Summary:

The GPT-2 is a new approach to the problem of text generation. It is based on the concept of a "text-generating machine", which is a computer program that generates text from
